# MTH3302: Méthodes probabilistes et statistiques pour l'I.A.
## Projet

- Vincent Haney
- Félix Pelletier
- Charles-François St-Cyr (275354)
- Mikael Vaillant (2029994)


### Librairies

In [3]:
import Pkg;
packages = ["CSV", "DataFrames", "Distributions", "Gadfly", "MLBase", "Random", "Statistics", "Colors", "GLM", "LinearAlgebra", "Combinatorics", "CategoricalArrays","Distances","NearestNeighbors"];
# peut aps être mis dans une fonction. using doit être top-level
for package in packages
    try
        @eval using $(Symbol(package))
    catch e
        Pkg.add(package)
        @eval using $(Symbol(package))
    end
end
Pkg.update()

include("./library/main.jl")
include("./library/normalize.jl")

    Updating registry at `C:\Users\vaill\.julia\registries\General.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Manifest.toml`


normalize (generic function with 9 methods)

### Chargement des données (partiel pour l'exploration)

In [4]:
data = CSV.read("./data/train.csv", DataFrame)
test = CSV.read("./data/test.csv", DataFrame)

# data[!, :volume] .= data.x .* data.y .* data.z

train, valid = partitionData(data, 0.75);

first(train, 5)

Row,ID,cut,color,clarity,depth,table,price,x,y,z
,Int64,String15?,String1,String7,Float64?,Float64,Int64,Float64,Float64?,Float64
1,2,Ideal,G,VVS2,62.7,57.0,1936,5.12,5.05,3.19
2,4,Ideal,G,VS2,61.7,57.0,720,4.41,4.38,2.71
3,5,missing,G,VS2,62.4,58.0,6301,6.41,6.48,4.02
4,7,Very Good,D,SI1,60.2,61.0,571,4.39,4.42,2.65
5,9,Fair,I,VS1,64.4,59.0,1229,5.3,5.2,3.38


### Visualisation des données

In [4]:
# plot(data, x = :price, Geom.histogram())

In [5]:
missing_counts = collect(count(ismissing,col) for col in eachcol(data))
println("Nombre de valeurs manquantes dans les différentes colonnes: ")
println(missing_counts)

zero_counts = [count(x -> x == 0, skipmissing(data[:, col])) for col in names(data)]
println("Nombre de valeurs nulles dans les différentes colonnes: ")
println(zero_counts)


Nombre de valeurs manquantes dans les différentes colonnes: 
[0, 3806, 0, 0, 2201, 0, 0, 0, 2200, 0]
Nombre de valeurs nulles dans les différentes colonnes: 
[0, 0, 0, 0, 13, 0, 0, 2, 1, 15]


In [42]:
function fillDataFromAverage(data::DataFrame; is_test::Bool = false)
    avgCut = mode(dropmissing(data, :cut).cut)
    avgColor = mode(dropmissing(data, :color).color)
    avgClarity = mode(dropmissing(data, :clarity).clarity)
    avgDepth = mean(dropmissing(data, :depth).depth)
    avgTable = mean(dropmissing(data, :table).table)
    avgX = mean(dropmissing(data, :x).x)
    avgY = mean(dropmissing(data, :y).y)
    avgZ = mean(dropmissing(data, :z).z)
    # avgVol = mean(dropmissing(data, :volume).volume)

    replace!(data.cut, missing => avgCut)
    replace!(data.color, missing => avgColor)
    replace!(data.clarity, missing => avgClarity)
    replace!(data.depth, missing => avgDepth)
    replace!(data.table, missing => avgTable)
    replace!(data.x, missing => avgX)
    replace!(data.y, missing => avgY)
    replace!(data.z, missing => avgZ)
    # replace!(data.volume, missing => avgVol)

    if (!is_test)
        avgPrice = mean(dropmissing(data, :price).price)
        replace!(data.price, missing => avgPrice)
    end
end

fillDataFromAverage (generic function with 1 method)

In [90]:
train₁ = deepcopy(train)
valid₁ = deepcopy(valid)
test₁ = deepcopy(test)

fillDataFromAverage(train₁)
fillDataFromAverage(valid₁)
fillDataFromAverage(test₁, is_test = true)

convertQualitativeToQuantitative(train₁, :color)
convertQualitativeToQuantitative(train₁, :cut)
convertQualitativeToQuantitative(train₁, :clarity)

convertQualitativeToQuantitative(valid₁, :color)
convertQualitativeToQuantitative(valid₁, :cut)
convertQualitativeToQuantitative(valid₁, :clarity)

convertQualitativeToQuantitative(test₁, :color)
convertQualitativeToQuantitative(test₁, :cut)
convertQualitativeToQuantitative(test₁, :clarity)

price_train = select(train₁, (:price))
price_true = select(valid₁, (:price))

train₁ = select(train₁, Not(["price","cut","ID","color","clarity","y","depth"]))
valid₁ = select(valid₁, Not(["price","cut","ID","color","clarity","y","depth"]))

for col in names(train₁[:,1:5])
    train₁[!, col] = (train₁[!, col] .- mean(train₁[!, col])) ./ std(train₁[!, col])
end

for col in names(valid₁[:,1:5])
    valid₁[!, col] = (valid₁[!, col] .- mean(valid₁[!, col])) ./ std(valid₁[!, col])
end



first(train₁,5)

Row,table,x,z,color: E,color: F,color: G,color: H,color: I,color: J,cut: Good,cut: Ideal,cut: Premium,cut: Very Good,clarity: IF,clarity: SI1,clarity: SI2,clarity: VS1,clarity: VS2,clarity: VVS1,clarity: VVS2
,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,-0.20882,-0.549893,-0.505263,-0.468337,-0.462992,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,-0.20882,-1.17977,-1.19172,-0.468337,-0.462992,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0.239207,0.594528,0.681735,-0.468337,-0.462992,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,1.58329,-1.19751,-1.27753,-0.468337,-0.462992,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
5,0.687233,-0.390206,-0.233541,-0.468337,-0.462992,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


#### Prédiction

In [73]:
# function knn(X_train,y_train,X_test,k)
#     n_test = size(X_test,1)
#     n_train = size(X_train, 1)
#     y_pred = zeros(n_test)

#     train₁_T2 = Matrix(X_train)'.* Matrix(X_train)'


#     for i in 1:n_test
#         # distances = pairwise(Euclidean(), X_train, X_test[i,:])
#         # println(size(distances))
#         sorted_indices = sortperm(sum(sqrt.(train₁_T2 .+ Matrix(X_test)'[:,i].*Matrix(X_test)'[:,i]),dims=1)[1,:])
#         k_nearest_indices = sorted_indices[1:k]
#         k_nearest_labels = y_train[k_nearest_indices]
#         y_pred[i] = mode(k_nearest_labels)
#     end

#     return y_pred
# end

In [96]:

data = Matrix(train₁)'
k = 4
point = Matrix(valid₁)'

kdtree = KDTree(data)
idxs, dists = knn(kdtree, point, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

RMSE₁₁ = sqrt(mean((prediction .- Matrix(price_true)).^2))



886.525208980294